
> [!NOTE]
> Timebox: 60 minutes (20 minutes of content | 40 minutes of lab work)
> 
> [Back to Agenda](./../README.md#agenda) | [Back to Start Steps](../module-0-setup/start.md) | [Up next Exercise 2](./../exercise-2/exercise-2.md)

# 🚀 Lab 3: Job Scheduling, Monitoring, and Debugging  

### 🏗️ What You'll Learn  
In this lab, you'll dive into essential Spark monitoring and debugging techniques to gain deep insights into your Spark applications.  

🔍 **Exploring the Spark UI**
- Detect task skews & utilization issues
- Track native execution
- Leverage rolling logs  

📊 **Spark Monitoring UI**  
- Analyze resource usage  
- Investigate concurrent jobs  
- View snapshots of your pipeline 
- Utilize notebook utilities

🛠️ **Live Application Debugging**
- Use print statements & logging  
- Set breakpoints 
- Analyze partitions (`getNumPartitions`)

🔥 **High Concurrency (HC) Monitoring**   
- Job-to-notebook mapping 
- REPL-level logging 

📡 **Emitting Spark Logs & Metrics**
- **Event Hub** for real-time monitoring.  
- **Blob Storage** for long-term retention and auditing.  

---

### 🖥️ Context: Microsoft Fabric Spark Monitoring  
Microsoft Fabric’s Spark monitoring offers a **web-UI-driven experience** with powerful built-in tools to:  
✔️ Track Spark applications in progress.  
✔️ Browse historical Spark activity.  
✔️ Analyze performance bottlenecks.  
✔️ Troubleshoot failures effectively.  

With multiple entry points, Spark monitoring ensures seamless access to job details, making it easier to optimize execution and resolve issues.  


### Preparation  

To get started, follow these steps:  

1. **Upload the dataset**  
   - Add the file [`online_retail.csv`](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/online_retail.csv) to your lakehouse.  

2. **Import the notebook**  
   - Load the [`SparkMonitoring-2025.ipynb`](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/SparkMonitoring-2025.ipynb) notebook into your environment.  

Once completed, you're ready to proceed with this lab ! 🚀 

### 3.1 Monitor Hub  

The **Monitor Hub** is the centralized portal for tracking Spark activities across different components. With a quick glance, you can:  

- View in-progress Spark applications triggered from **Notebooks, Spark Job Definitions, and Pipelines**.  
- Search and filter Spark applications  
- Drill down into Spark execution details for deeper insights.  

#### Accessing the Monitor Hub  

To open the Monitor Hub from the **Fabric portal**, follow these steps:  

1. Navigate to the **left sidebar**.  
2. Click on **Monitor** to access the monitoring pane.  

Here, you can explore your Spark jobs effortlessly.  

![Monitor Hub Navigation](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/monitoring-hub-in-the-left-side-navigation-bar.png?raw=true)  

Stay on top of your Spark applications with ease! 🚀  

### 3.2 Item Recent Runs  

The **Recent Runs** feature helps you track your current and past activities for specific items. It provides key insights, including:  

- **Submitter** – Who initiated the job  
- **Status** – Whether the job is running, completed, or failed  
- **Duration** – How long the job took  
- **Other details** – Additional execution metadata  

#### Accessing Recent Runs  

To view recent runs, follow these steps:  

1. Open the **Microsoft Fabric** homepage.  
2. Select the **workspace** where your job is running.  
3. Locate the relevant **Spark Job Definition, Notebook, or Pipeline**.  
4. Open the **context menu** for the item and select **Recent Runs**.  
5. The **Recent Runs** pane will display past execution details for easy monitoring.  

![Recent Runs List](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/recent-runs-list.png?raw=true)  

### 3.3 Notebook Contextual Monitoring  

**Notebook Contextual Monitoring** brings everything you need into one place—**authoring, monitoring, and debugging Spark jobs** seamlessly. With this feature, you can:  

- **Track Spark job progress** in real time.  
- **View execution details** like tasks and executors.  
- **Access Spark logs** directly at the **Notebook cell level**.  
- Get **real-time insights** from the built-in **Spark Advisor**, which provides code recommendations, execution analysis, and error debugging.  

---

#### 3.3.1 Monitor Spark Job Progress  

A **real-time progress bar** provides instant visibility into Spark job execution within each Notebook cell. This helps you:  

✅ Monitor the status of each Spark job.  
✅ Track task execution across different stages.  
✅ Troubleshoot efficiently with live updates.  

##### Try it out!  

🚀 **Run the cell below** and expand the **Spark Jobs** section to view real-time status updates and task progress across execution stages.  

![Spark Monitoring Progress](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/spark-monitor-progress.png?raw=true)  

Stay on top of your Spark jobs like a pro! 💡🔥  

In [1]:
val df = spark.read.format("csv").option("header","true").load("Files/online_retail.csv")
display(df)

StatementMeta(, c8f3477c-cd91-404a-91e9-c547f8e1e160, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3aa98641-d25e-474f-9345-e592e0569065)


df: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


#### 3.3.2 Monitor Resource Usage  

The **Executor Usage Graph** provides a **visual representation** of **Spark job executors and resource consumption**.  

- Currently, only the runtime information of spark 3.4 and above will display this feature. 
- Displays **real-time resource usage** per code cell.  
- Helps optimize performance by analyzing **executor allocation**.  

🔹 **Try it out:** Click on the **Resources** tab to view the line chart for your code cell’s resource usage.  

![Resource Usage Graph](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/resource.png?raw=true)  

With **Notebook Contextual Monitoring**, you can **debug faster, optimize better, and gain deeper insights** into your Spark jobs—all in one place! 🚀  

#### 3.4.3 Hands-on Exercise: Monitoring Resource Utilization  

##### **Step 1: First, run the following preparation code:**    

In [3]:
// Create some fake data and task definition for the demostration below.
import java.util.concurrent.TimeUnit
import scala.util.Random

val data = Array.fill(32)(Random.nextInt(32))

def calculation(iter: Iterator[Int]): Iterator[Int] = {
    val sum = iter.map(_ => 1).sum
    TimeUnit.SECONDS.sleep(10*sum)
    Iterator(sum)
}

StatementMeta(, c8f3477c-cd91-404a-91e9-c547f8e1e160, 5, Finished, Available, Finished)

import java.util.concurrent.TimeUnit
import scala.util.Random
data: Array[Int] = Array(20, 14, 6, 17, 11, 9, 24, 30, 2, 19, 21, 19, 25, 22, 8, 30, 2, 30, 22, 19, 23, 19, 31, 13, 7, 25, 4, 5, 20, 12, 2, 28)
calculation: (iter: Iterator[Int])Iterator[Int]


##### **Step 2: Run the cell below.**
Now, observe the Resource Usage tab.
Notice that the job does not utilize all available cores, meaning we should increase parallelism.

In [4]:
// Run a simple job.
// In the resource usage tab, we can see that the job did not use all of available cores. It means that we should increase parallelism to fully use the executor resource.
val rdd1 = sc.parallelize(data, 10)
val results1 = rdd1.mapPartitions(calculation).collect()
println(results1)

StatementMeta(, c8f3477c-cd91-404a-91e9-c547f8e1e160, 6, Finished, Available, Finished)

[I@6b4d7327
rdd1: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[54] at parallelize at <console>:33
results1: Array[Int] = Array(3, 3, 3, 3, 4, 3, 3, 3, 3, 4)


##### **Step 3: Now, run the same job with a higher number of partitions (32) to take advantage of all available cores.**
You will see that all cores are used.

In [5]:
// Run the same job again, but with more partitions.
// Now we can see that all cores are used.
val rdd2 = sc.parallelize(data, 32)
val results2 = rdd2.mapPartitions(calculation).collect()
println(results2)

StatementMeta(, c8f3477c-cd91-404a-91e9-c547f8e1e160, 7, Finished, Available, Finished)

[I@598ef685
rdd2: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[56] at parallelize at <console>:33
results2: Array[Int] = Array(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)


#### 3.3.3 Access Spark Real-Time Logs  

Need to debug issues quickly? **Contextual Monitoring** brings **Spark logs** directly into your notebook! 🛠️  

- Easily locate **exceptions, warnings, and errors**  
- **Search or filter logs** for faster troubleshooting  
- Click on the **Log tab** in the cell to explore logs in real-time  

![Real-Time Logs](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/resource.png?raw=true)

### 3.4 Spark job definition inline monitoring

The Spark job definition Inline Monitoring feature allows you to view Spark job definition submission and run status in real-time, as well as view the Spark job definition's past runs and configurations. You can navigate to the Spark application detail page to view more details.

#### 3.4.1 Spark job definition inline monitoring
The Spark job definition inline monitoring feature allows you to view Spark job definition submission and run status in real-time. You can also view the Spark job definition's past runs and configurations and navigate to the Spark application detail page to view more details.
![](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/spark-job-definition-inline-monitoring.png?raw=true)


#### 3.4.2 Spark job definition item view in workspace
You can access the job runs associated with specific Spark job definition items by using the Recent runs contextual menu on the workspace homepage.
![](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/spark-job-definition-artifact-view-in-workspace.png?raw=true)


### 3.5 Pipeline Spark activity inline monitoring
For Pipeline Spark Activity Inline Monitoring, deep links have been built into the Notebook and Spark job definition activities within the Pipeline. You can view Spark application execution details, the respective Notebook and Spark job definition snapshot, and access Spark logs for troubleshooting. If the Spark activities fail, the inline error message is also available within Pipeline Spark activities.

[Snapshot](https://learn.microsoft.com/en-us/fabric/data-engineering/media/spark-detail-monitoring/related-items.png)

### 3.6 OSS based Spark UI/Spark History Server

Use extended Apache Spark history server to debug and diagnose Apache Spark applications.

#### 3.6.1 Open the Spark web UI from progress indicator notebook
When an Apache Spark job is triggered, the button to open Spark web UI is inside the More action option in the progress indicator. In cell #5 that you just ran, select Spark web UI on the right and wait for a few seconds, then the Spark UI page appears.
![](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/spark-web-ui-in-the-progress-indicator-notebook.png?raw=true)

#### 3.6.2 Open the Spark web UI from Apache Spark application detail page
The Spark web UI can also be opened through the Apache Spark application detail page. Select Monitor on the left side of the page, and then select an Apache Spark application. The detail page of the application appears.
![](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/spark-web-ui-from-application-detail-page.png?raw=true)

#### 3.6.2 Advanced enhancement on Spark UI/Spark History Server
[Graph tab in Apache Spark history server](https://learn.microsoft.com/en-us/fabric/data-engineering/media/apache-spark-history-server/apache-spark-graph-job-id.png)


### 3.7 Run series
The Apache Spark run series automatically categorizes your Spark applications based on recurring pipeline activities, manual notebook runs, or Spark job runs from the same notebook or Spark job definition.

The run series feature illustrates the duration trend and data input or output trend for each Spark application instance. It automatically scans the run series, detects anomalies, and provides detailed views for individual Spark applications.

You can access the monitor run series feature from the **Monitoring hub**'s historical view and **Recent runs** panel:
|![](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/access-run-series-from-historical-view.png?raw=true)
![](./access-run-series-from-recent-run.png)

### 3.8 Spark advisor
The Apache Spark advisor analyzes commands and code run by Apache Spark and displays real-time advice for Notebook runs. The Apache Spark advisor has built-in patterns to help users avoid common mistakes. It offers recommendations for code optimization, performs error analysis, and locates the root cause of failures.

![](https://github.com/voidfunction/FabCon25SparkWorkshop/blob/main/module-3-scheduling-monitoring-debugging/_media/errors.png?raw=true)

#### 3.8.1. Error Type

Advice with Error type usually would be within the cell with failed execution. But not every failed cell will have an error type advice.
For this case, usually the tsg will include how you could make your cell execute successfully.

##### 3.8.1a Spark Job Failed/File Not Found

In [8]:
%%spark
def test(): Unit = {
    val data = 1 to 100000
    val inputRdd = sc.parallelize(data, 3)
    inputRdd.map(number => {
        // Thread.sleep(1000)
        if (number % 5 == 0) {
            throw new java.io.FileNotFoundException("Not found issues")
        }

        if (number % 3 == 0) {
            throw new RuntimeException("Authenticate failure issues")
        }
    }).count()
}

val df = spark.read.format("csv").option("header","true").load("Files/online_retail.csv")
display(df)

test()

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cf249f21-38a4-469b-aaa6-b2bd468fa381)

Error: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 23.0 failed 4 times, most recent failure: Lost task 1.3 in stage 23.0 (TID 69) (vm-65011780 executor 2): java.io.FileNotFoundException: Not found issues

##### 3.8.1b Path Already exists Error, and solution is provided as: To overwrite the existing file, set mode("overwrite") on the DataFrameWriter

In [10]:
%%spark
df.write.csv("Files/catalogparquet2csv")

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 12, Finished, Available, Finished)

Error: org.apache.spark.sql.AnalysisException: [PATH_ALREADY_EXISTS] Path abfss://4cb9b656-c8f8-485e-a151-e81bb913abc8@msit-onelake.dfs.fabric.microsoft.com/eb9433f1-ec4d-4683-89bd-49b5c4ee819b/Files/catalogparquet2csv already exists. Set mode as "overwrite" to overwrite the existing path.

##### 3.8.1c After adding overwrite as saveMode, execution succeeded.

In [11]:
%%spark
import org.apache.spark.sql.SaveMode
df.write.mode(SaveMode.Overwrite).csv("Files/catalogparquet2csv")

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 13, Finished, Available, Finished)

import org.apache.spark.sql.SaveMode


### 3.8.2. Warning Type

##### Advice with warning type usually would not result in a failed execution, but some of the passed in confs/plugins might not work as expected. Or there might exist perf issue like skew.
The message would mostly about how to make it work as you expected

##### 3.8.2a Data Skew/Time Skew

In [ ]:
%%spark
import org.apache.spark.SparkContext 
import scala.util.Random 
def testDataSkew(sc: SparkContext): Unit = {
    val numMappers = 400
    val numKVPairs = 10000
    val valSize = 256
    val numReducers = 200
    val biasPct = 0.4
    val biasCount = numKVPairs * biasPct
    for (i <- 1 to 2) {
      val query = sc.parallelize(0 until numMappers, numMappers).flatMap { p =>
        val ranGen = new Random
        val arr1 = new Array[(Int, Array[Byte])](numKVPairs)
        for (i <- 0 until numKVPairs) {
          val byteArr = new Array[Byte](valSize)
          ranGen.nextBytes(byteArr)
          var key = ranGen.nextInt(Int.MaxValue)
          if(i <= biasCount) {
            key = 1
          }
          arr1(i) = (key, byteArr)
        }
        arr1
      }.groupByKey(numReducers)
      // Enforce that everything has been calculated and in cache
      // scalastyle:off println
      println(query.count())
      // scalastyle:on println
      Thread.sleep(1000)
    }
  }


testDataSkew(sc)

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 14, Finished, Available, Finished)

2398241
2398279
import org.apache.spark.SparkContext
import scala.util.Random
testDataSkew: (sc: org.apache.spark.SparkContext)Unit


In [ ]:
%%spark
import org.apache.spark.TaskContext
val res = spark.range(0, 1000 * 1000, 1).repartition(200).map { x =>
  if (TaskContext.get.attemptNumber == 0 && TaskContext.get.partitionId < 8) {
    Thread.sleep(30)
  }
  x
}.repartition(200).map { x =>
  x
}
res.distinct().count()

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 26, Finished, Available, Finished)

import org.apache.spark.TaskContext
res: org.apache.spark.sql.Dataset[Long] = [value: bigint]
res46: Long = 1000000


##### 3.8.2b Hint related warning

##### Hint Not recognized

create below table first:

In [ ]:
spark.sql("CREATE TABLE t1 (str STRING) USING parquet")
spark.sql("CREATE TABLE t2 (str STRING) USING parquet")
spark.sql("CREATE TABLE t3 (str STRING) USING parquet")

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 16, Finished, Available, Finished)

Error: org.apache.spark.sql.catalyst.analysis.TableAlreadyExistsException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `spark_catalog`.`monitoringlh`.`t1` because it already exists.

In [ ]:
spark.sql("SELECT /*+ testtest */ * FROM t1")

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 17, Finished, Available, Finished)

res30: org.apache.spark.sql.DataFrame = [str: string]


##### Hint Overriden

In [ ]:
spark.sql("SELECT /*+ BROADCAST(t1), MERGE(t1, t2) */ * FROM t1 INNER JOIN t2 ON t1.str = t2.str")

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 18, Finished, Available, Finished)

res31: org.apache.spark.sql.DataFrame = [str: string, str: string]


##### Couldn't found relations

In [ ]:
spark.sql("SELECT /*+ BROADCAST(test1) */ * FROM t1 INNER JOIN t2 ON t1.str = t2.str")

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 19, Finished, Available, Finished)

res32: org.apache.spark.sql.DataFrame = [str: string, str: string]


#### 3.8.3. Info Type

##### Advice with Info type usually mean use this way might improve the execution.

##### 3.8.3a RandomSplit might introduce inconsistency

In [18]:
%%spark
val rdd = sc.parallelize(1 to 1000000)
val rdd2 = rdd.repartition(64)
val Array(train, test) = rdd2.randomSplit(Array(70, 30), 1)
train.takeOrdered(10)

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 20, Finished, Available, Finished)

rdd: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[152] at parallelize at <console>:31
rdd2: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[156] at repartition at <console>:31
train: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[157] at randomSplit at <console>:32
test: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[158] at randomSplit at <console>:32
res33: Array[Int] = Array(2, 3, 4, 5, 6, 8, 9, 10, 13, 14)


#### below data is not stable during multi runs, even there is overlap between dataset: train and test.

In [19]:
%%spark
train.takeOrdered(10)

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 21, Finished, Available, Finished)

res34: Array[Int] = Array(2, 3, 4, 5, 6, 8, 9, 10, 13, 14)


##### 3.8.3b DivisionExprAdvise

Division expressions can be reduced and shuffled to reduce rounding error propagation. 

In this advise, we check the optimized logical plan to get the expressions below: 

A / B * C, A * (B / C), A / B / C, A / (B / C) and if A’s data type is Double. 

These expressions can be converted to sematic equivalent expressions: Like A / B / C into A / (B * C) 

In [20]:
val exprA = "(CAST(id AS DOUBLE) + 1.0D)"
val exprB = "(CAST(id AS DOUBLE) + 2.0D)"
val exprC = "(CAST(id AS DOUBLE) * 3.0D)"
val expr = s"${exprA} / ${exprB} * ${exprC}"
val df = spark.range(1)
        .selectExpr(expr)
        .as("result")
      df.collect()

StatementMeta(, a86d7c03-092e-482f-b3ab-ccd21d3b0522, 22, Finished, Available, Finished)

exprA: String = (CAST(id AS DOUBLE) + 1.0D)
exprB: String = (CAST(id AS DOUBLE) + 2.0D)
exprC: String = (CAST(id AS DOUBLE) * 3.0D)
expr: String = (CAST(id AS DOUBLE) + 1.0D) / (CAST(id AS DOUBLE) + 2.0D) * (CAST(id AS DOUBLE) * 3.0D)
df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [(((CAST(id AS DOUBLE) + 1.0) / (CAST(id AS DOUBLE) + 2.0)) * (CAST(id AS DOUBLE) * 3.0)): double]
res35: Array[org.apache.spark.sql.Row] = Array([0.0])


### 3.9 Collect log with Azure log analystics, Azure Storage account and Azure eventhub

The Fabric Apache Spark diagnostic emitter extension is a library that enables Apache Spark applications to emit logs, event logs, and metrics to multiple destinations, including Azure log analytics, Azure storage, and Azure event hubs.

#### 3.9.1 Configure with Log Analytics Workspace ID and Key

```
spark.synapse.diagnostic.emitters: <EMITTER_NAME>
spark.synapse.diagnostic.emitter.<EMITTER_NAME>.type: "AzureLogAnalytics"
spark.synapse.diagnostic.emitter.<EMITTER_NAME>.categories: "Log,EventLog,Metrics"
spark.synapse.diagnostic.emitter.<EMITTER_NAME>.workspaceId: <LOG_ANALYTICS_WORKSPACE_ID>
spark.synapse.diagnostic.emitter.<EMITTER_NAME>.secret: <LOG_ANALYTICS_WORKSPACE_KEY>
spark.fabric.pools.skipStarterPools: "true" //Add this Spark property when using the default pool.
```

#### 3.9.2 Configure with Azure Storage URI and Access key

```
spark.synapse.diagnostic.emitters: MyStorageBlob
spark.synapse.diagnostic.emitter.MyStorageBlob.type: "AzureStorage"
spark.synapse.diagnostic.emitter.MyStorageBlob.categories: "DriverLog,ExecutorLog,EventLog,Metrics"
spark.synapse.diagnostic.emitter.MyStorageBlob.uri:  "https://<my-blob-storage>.blob.core.windows.net/<container-name>/<folder-name>"
spark.synapse.diagnostic.emitter.MyStorageBlob.auth: "AccessKey"
spark.synapse.diagnostic.emitter.MyStorageBlob.secret: <storage-access-key>
spark.fabric.pools.skipStarterPools: "true" //Add this Spark property when using the default pool.
```

#### 3.9.3 Configure with Azure Event Hubs Connection String

```
spark.synapse.diagnostic.emitters: MyEventHub
spark.synapse.diagnostic.emitter.MyEventHub.type: "AzureEventHub"
spark.synapse.diagnostic.emitter.MyEventHub.categories: "Log,EventLog,Metrics"
spark.synapse.diagnostic.emitter.MyEventHub.secret: <connection-string>
spark.fabric.pools.skipStarterPools: "true" //Add this Spark property when using the default pool.
```
